In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StringType, IntegerType, FloatType
from pyspark.sql.window import Window

# Create a SparkSession
spark = SparkSession.builder \
.appName("ReadExcelWithHeader") \
.config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
.getOrCreate()

excel_file_path = "sampleCur.xlsx"

In [22]:
df = spark.read \
.format("com.crealytics.spark.excel") \
.option("header", "true") \
.option("inferSchema", "true") \
.option("dataAddress", "'0'!A7") \
.load(excel_file_path)

# Show the DataFrame
df.show()

+----+---------------------+----------------------------------+--------------------+------------------------------+----+---+--------------------+-----+----------------------------+------------------------------------------------+------------------+
|   #|Number of Generations|Club Member (First name/Last name)|                 _c3|CNV (Cumulated Network Volume)|Rank|  P|PV (Personal Volume)|  PVG|MCR (Maximum Confirmed Rank)|Number of months with zero Volume Points accrued|Club Member Status|
+----+---------------------+----------------------------------+--------------------+------------------------------+----+---+--------------------+-----+----------------------------+------------------------------------------------+------------------+
| 1.0|                  .0 |                         6111685.0|         Галюк Елена|                         > 500|   E|   |                 100|  230|                         E/1|                                                |                  |
| 2.

In [57]:
from pyspark.sql import functions as F

@udf(returnType=IntegerType())
def parseGen(dotted: str) -> int:
    return int(dotted.split(".")[-1])

excel2Field = {}
excel2Field["#"] = "rownum"
excel2Field["Number of Generations"] = "gen"
excel2Field["Club Member (First name/Last name)"] = "Id"
excel2Field["_c3"] = "Name"
excel2Field["CNV (Cumulated Network Volume)"] = "cnv"
excel2Field["Rank"] = "Rank"
excel2Field["P"] = "Pending"
excel2Field["PV (Personal Volume)"] = "Personal"
excel2Field["PVG"] = "Group"
excel2Field["MCR (Maximum Confirmed Rank)"] = "maxrank"
excel2Field["Number of months with zero Volume Points accrued"] = "zeroMonths"
excel2Field["Club Member Status"] = "status"

df_remapColumns = df.withColumnsRenamed(excel2Field)
df_dropRownum = df_remapColumns.drop(df_remapColumns.rownum)
df_recastId = df_dropRownum.withColumn("Id", df_dropRownum.Id.cast(IntegerType()))

df_digitizeGen = df_recastId.withColumn("gen", parseGen(df_recastId.gen))

df_digitizeGen.show()


+---+-------+--------------------+-----+----+-------+--------+-----+-------+----------+------+
|gen|     Id|                Name|  cnv|Rank|Pending|Personal|Group|maxrank|zeroMonths|status|
+---+-------+--------------------+-----+----+-------+--------+-----+-------+----------+------+
|  0|6111685|         Галюк Елена|> 500|   E|       |     100|  230|    E/1|          |      |
|  1|6190205|       Галюк Дмитрий|> 500|   D|       |        |     |  SD/28|         1|     m|
|  2|6202838| Tabolich Anastasiya|> 500|  PM|       |        |     |       |        12|     m|
|  2|6866923|Фалавандишвили Натия|> 500|  PM|       |        |    7|   D/28|         7|     m|
|  3|6959087|    Мурванидзе Луиза|> 500|  PM|       |       7|     |       |          |     m|
|  4|7309266|  Murvanidze Violeta|135.5|    |       |        |     |       |         3|     m|
|  3|7575828|Фалавандишвили Ти...|   35|    |       |        |     |       |         7|     m|
|  2|7688719|       Konidala Anna|> 500|  PM|     

In [62]:
levelId = {}
@udf(returnType=IntegerType())
def id_gen_2_parent(id, gen):
    levelId[gen] = id
    return levelId[gen-1] if gen > 0 else None

df_withParent: DataFrame = df_digitizeGen.withColumn("ParentId", id_gen_2_parent(df_digitizeGen.Id, df_digitizeGen.gen))
df_withParent.show()

+---+-------+--------------------+-----+----+-------+--------+-----+-------+----------+------+--------+
|gen|     Id|                Name|  cnv|Rank|Pending|Personal|Group|maxrank|zeroMonths|status|ParentId|
+---+-------+--------------------+-----+----+-------+--------+-----+-------+----------+------+--------+
|  0|6111685|         Галюк Елена|> 500|   E|       |     100|  230|    E/1|          |      |    NULL|
|  1|6190205|       Галюк Дмитрий|> 500|   D|       |        |     |  SD/28|         1|     m| 6111685|
|  2|6202838| Tabolich Anastasiya|> 500|  PM|       |        |     |       |        12|     m| 6190205|
|  2|6866923|Фалавандишвили Натия|> 500|  PM|       |        |    7|   D/28|         7|     m| 6190205|
|  3|6959087|    Мурванидзе Луиза|> 500|  PM|       |       7|     |       |          |     m| 6866923|
|  4|7309266|  Murvanidze Violeta|135.5|    |       |        |     |       |         3|     m| 6959087|
|  3|7575828|Фалавандишвили Ти...|   35|    |       |        |  